# Blackjack Program
### Ruleset with soft Aces, Blackjacks, and no splits, double-downs, or insurance.
By Paul Zee-Cheng

November 31, 2021

In [1]:
class Card:
    '''
    Class function to create the Cards necessary to play blackjack.
    '''
    def __init__(self,suit,rank):
        
        self.suit = suit
        self.rank = rank
        
    def __str__(self):
        
        return self.rank + " of " + self.suit

In [2]:
class Deck:
    '''
    Class function to assemble all the cards into a Deck that can be dealt, shuffled, etc.
    '''
    def __init__(self):
        
        self.card_deck = []
        
        for suit in suits:
            for rank in ranks:
                self.card_deck.append(Card(suit,rank))
                
    def __str__(self):
        '''
        Print string to help debug code.
        '''
        deck_comp = ''
        
        for card in self.card_deck:
            deck_comp += '\n ' + card.__str__()
        return 'The deck has:' + deck_comp

    def shuffle(self):
        
        random.shuffle(self.card_deck)
        
    def deal(self):

        single_card = self.card_deck.pop()
        return single_card

In [3]:
class Hand:
    '''
    Class function to create a player AND a dealer, who each holds a hand of cards.
    '''
    def __init__(self):
        self.hand_cards = []
        self.value = 0
        self.aces = 0
        
        #I created all these attributes to run a more complex game.
        self.blackjack = False
        self.adjusted_value = 0
        self.adjusted_aces = 0
        
    def add_card(self,new_card):
        '''
        Adds new card, but also updates all hand attributes with each new card.
        '''
        self.hand_cards.append(new_card)
        self.value += values[new_card.rank]
        if new_card.rank == "Ace":
            self.aces += 1
        if self.value == 21 and len(self.hand_cards) == 2:
            self.blackjack = True
        
    def adjust_for_ace(self):
        '''
        To calculate adjusted value of hand - unlike proposed examples, can calculate each time method is run.
        '''
        self.adjusted_value = self.value
        self.adjusted_aces = self.aces
        
        while self.adjusted_value > 21:
            if self.adjusted_aces > 0:
                self.adjusted_value = self.adjusted_value - 10
                self.adjusted_aces = self.adjusted_aces - 1
            else:
                break
        return self.adjusted_value
            

    def __str__(self):
        '''
        To print out all info - not required program, but very useful to debug.
        '''
        hand_comp = ''
        for card in self.hand_cards:
            hand_comp += '\n ' + card.__str__()
        return hand_comp + "\nNumber of Aces: " + str(self.aces) + "\nRaw Value of Hand: " + str(self.value) + "\nBlackjack:" + str(self.blackjack)
   

In [4]:
class Chips:
    '''
    Class function to create chips necessary for the player to wager to win the game.
    '''
    def __init__(self):
        self.total = 100
        self.bet = 0
    
    def win_bet(self):
        self.total += self.bet
    
    def lose_bet(self):
        self.total -= self.bet
        
    def __str__(self):
        return f"Player Chips: ${self.total}"

In [5]:
def take_bet(player_chips):
    '''
    Function to take bets.
    '''
    while True:
        
        try:
            player_chips.bet = int(input("How much will you wager?"))
            
        except ValueError:
            print('Sorry, please enter an integer!')
        
        else:
            
            if player_chips.bet > player_chips.total:
                print("Sorry, your bet can't exceed $",player_chips.total)
            else:
                break            

In [6]:
def hit(deck,hand):
    '''
    Function to define dealing one card - NOTE that this code block does not adjust for aces, as there
    is a separate function to do so at any time.
    '''    
    hand.add_card(deck.deal())

In [7]:
def hit_or_stand(deck,hand):
    '''
    Function to control the gameplay While loop (via playing variable) and deal out cards to player.
    '''
    global playing #To control Player's While Loop.
    while True:

        hit_or_stand = input("Do you want to Hit, or Stand? (Type H or S)").upper()

        if hit_or_stand not in ["H","S"]:
            print("Sorry, I don't understand!")
            continue
    
        elif hit_or_stand == "H":
            hit(deck,hand)
            print("Hit - dealing a card!")
            
        elif hit_or_stand == "S":            
            playing = False
            print("Player stands. Dealer is playing.")
            
        break

In [8]:
'''
Functions to display the 'screen' for the player - an intro screen, the cards dealt to player, and dealer's hand.
'''

def show_none(player_chips):
    print("\nDealer's Hand:")
    print(" * <No Card>")
    print(" * <No Card>")
    print("-------------------")
    print("     Hand Value Unknown!")
    print("\n\nPlayer's Hand:")
    print(" * <No Card>")
    print(" * <No Card>")
    print("-------------------")
    print("     Hand Value Unknown!\n")
    print("     Player Chips:",player_chips.total)    
    
    
def show_some(player_hand,dealer_hand,player_chips):
        
    print("\nDealer's Hand:")
    print(" * <Card Hidden>")
    print(" *",dealer_hand.hand_cards[1])
    print("-------------------")
    print("     Hand Value Unknown!")
    print("\n\nPlayer's Hand:",*player_hand.hand_cards,sep = "\n * ")
    player_hand.adjust_for_ace()
    print("-------------------")
    if player_hand.blackjack == True:
        print("     BLACKJACK!")
    print("     Current Hand Value:",player_hand.adjusted_value)
    print("     Player Bet:",player_chips.bet)

    
def show_all(player_hand,dealer_hand):
        
    print("\nDealer's Hand:",*dealer_hand.hand_cards,sep = "\n * ")
    dealer_hand.adjust_for_ace()
    print("-------------------")
    print("     Hand Value:",dealer_hand.adjusted_value)
    print("\n\nPlayer's Hand:",*player_hand.hand_cards,sep = "\n * ")
    player_hand.adjust_for_ace()
    print("-------------------")
    print("     Hand Value:",player_hand.adjusted_value)

In [9]:
'''
To run all different win conditions - blackjacks and regular wins.
Will adjust player chips, print statement, and return True if applied.
'''

# The first three functions will run before the dealer receives their cards - hence using the playing variable.

def player_busts(player_hand,player_chips):
    
    global playing
    player_hand.adjust_for_ace()
    if player_hand.adjusted_value > 21:
        player_chips.lose_bet()
        print(f"Player busts! You lose ${player_chips.bet}.")
        playing = False
        return playing
    else:
        pass

def player_blackjack(player_hand,dealer_hand,player_chips):

    global playing
    if player_hand.blackjack == True and dealer_hand.blackjack == False:
        player_chips.win_bet()
        input("Blackjack! Can the dealer match you? Press any key to find out.")
        clear_output()
        show_all(player_hand,dealer_hand)
        print(f"No! You win ${player_chips.bet}.")
        playing = False
        return playing
    else:
        pass
    
def blackjack_push(player_hand,dealer_hand,player_chips):
    
    if player_hand.blackjack == True and dealer_hand.blackjack == True:
        input("Blackjack! Can the dealer match you? Press any key to find out.")
        show_all(player_hand,dealer_hand)
        print(f"Yes - Dealer and Player tie. Your bet of ${player_chips.bet} is returned to you.")
        playing = False
        return playing
    else:
        pass
    
#The following scenarios run after the dealer gets their cards

def player_wins(player_hand,dealer_hand,player_chips):

    player_hand.adjust_for_ace()
    dealer_hand.adjust_for_ace()
    if player_hand.adjusted_value > dealer_hand.adjusted_value:
        player_chips.win_bet()
        print(f"Player wins! You win ${player_chips.bet}.")
        return
    else:
        pass

def dealer_busts(dealer_hand,player_chips):

    dealer_hand.adjust_for_ace()
    if dealer_hand.adjusted_value > 21:
        player_chips.win_bet()
        print(f"Dealer busts! You win ${player_chips.bet}.")
        return
    else:
        pass
    
def dealer_wins(player_hand,dealer_hand,player_chips):
    
    player_hand.adjust_for_ace()
    dealer_hand.adjust_for_ace()
    if player_hand.adjusted_value < dealer_hand.adjusted_value and dealer_hand.adjusted_value <= 21:
        player_chips.lose_bet()
        print(f"Dealer wins! You lose ${player_chips.bet}.")
        return
    else:
        pass
    
def push(player_hand,dealer_hand,player_chips):

    player_hand.adjust_for_ace()
    dealer_hand.adjust_for_ace()
    if player_hand.adjusted_value == dealer_hand.adjusted_value and dealer_hand.blackjack == False:
        print(f"Dealer and Player tie! Your bet of ${player_chips.bet} is returned to you.")
        return
    else:
        pass
    
def dealer_blackjack(player_hand,dealer_hand,player_chips):
    
    if player_hand.blackjack == False and dealer_hand.blackjack == True:
        player_chips.lose_bet()
        clear_output()
        show_all(player_hand,dealer_hand)
        print(f"Dealer has blackjack! You lose ${player_chips.bet}. Press any key to continue.")
        return
    else:
        pass

In [10]:
def replay():
    '''
    Variant code from my prior tic-tac-toe game - to see if player wishes to runs the game again.
    '''
    replay_answer = 'wrong'
    
    while replay_answer != "Y" or replay_answer != "N":
        replay_answer = input('Do you want to play again? (Y or N)')
        replay_answer = replay_answer.upper()

        if replay_answer not in ["Y","N"]:
            print("Sorry, I don't understand!")
        if replay_answer == "Y":
            return True
        if replay_answer == "N":
            return False

In [11]:
def dealer_turn_test(player_hand,dealer_hand):
    '''
    My code, outside of proposed examples - logic function necessary to test if it's the dealer's turn to go. Because
    of the more complicated Blackjack hand conditions created with this more complex game system,
    we can't just pass to the dealer if player does NOT bust, due to additional win conditions I created.
    '''
    player_hand.adjust_for_ace()

    if player_hand.adjusted_value > 21:
        return False
    elif player_hand.blackjack == True and dealer_hand.blackjack == False:
        return False
    elif player_hand.blackjack == True and dealer_hand.blackjack == True:
        return False
    else:
        return True

In [12]:
print("Welcome to Blackjack! Get as close as you can to 21 without going over!\n\
Dealer hits until they reach 17. Aces count as 1 or 11.\n\
Dealer plays a Soft 17, no splits, double-downs, or insurance allowed.\n\n")

p_chips = Chips()
print(p_chips)
print("Can you double your money?!?") # No actual reward for player doubling money - just incentive to add some stakes to game.

import random
from IPython.display import clear_output

suits = ('Hearts', 'Diamonds', 'Spades', 'Clubs')
ranks = ('Two', 'Three', 'Four', 'Five', 'Six', 'Seven', 'Eight', 'Nine', 'Ten', 'Jack', 'Queen', 'King', 'Ace')
values = {'Two':2, 'Three':3, 'Four':4, 'Five':5, 'Six':6, 'Seven':7, 'Eight':8, 'Nine':9, 'Ten':10, 'Jack':10,
         'Queen':10, 'King':10, 'Ace':11}

input("Press any key to play!")

while True:
        
    g_deck = Deck()
    g_deck.shuffle()

    d_hand = Hand()
    d_hand.add_card(g_deck.deal())
    d_hand.add_card(g_deck.deal())

    p_hand = Hand()
    p_hand.add_card(g_deck.deal())
    p_hand.add_card(g_deck.deal())    
    
    clear_output()
    show_none(p_chips)
    
    take_bet(p_chips)
    
    clear_output()
    show_some(p_hand,d_hand,p_chips)
    
    playing = True
    
    while playing:

        hit_or_stand(g_deck,p_hand)
        
        clear_output()
        show_some(p_hand,d_hand,p_chips)
        
        player_busts(p_hand,p_chips)
        player_blackjack(p_hand,d_hand,p_chips)
        blackjack_push(p_hand,d_hand,p_chips)

    if dealer_turn_test(p_hand,d_hand) == True:
    
        input("It's the Dealer's turn. Press any key to see their cards.")
        
        dealer_hitting = True
        
        while dealer_hitting == True:
                        
            if d_hand.value >= 17 and d_hand.value <= 21:
                dealer_hitting = False
            
            elif d_hand.value < 17:
                hit(g_deck,d_hand)
                dealer_hitting = True
            
            else: #(for when d_hand.value > 21)
                d_hand.adjust_for_ace()
                
                if d_hand.adjusted_value >= 17:
                    dealer_hitting = False
                
                if d_hand.adjusted_value < 17:
                    hit(g_deck,d_hand)
                    dealer_hitting = True

        clear_output()
        show_all(p_hand,d_hand)
        
        # Proposed examples had logic outside of win condition functions - not clear and difficult to read.
        # My code includes logic below, seems easier to understand win conditions.
        player_wins(p_hand,d_hand,p_chips)
        dealer_busts(d_hand,p_chips)
        dealer_wins(p_hand,d_hand,p_chips)
        push(p_hand,d_hand,p_chips)
        dealer_blackjack(p_hand,d_hand,p_chips)

    print(p_chips)
    
    if p_chips.total < 1:
        print("You've busted out!")
        break
        
    if replay() == False:
        print(f"Your chip payout: ${p_chips.total}!")
        break


Dealer's Hand:
 * Six of Clubs
 * Four of Hearts
 * Two of Clubs
 * Eight of Spades
-------------------
     Hand Value: 20


Player's Hand:
 * Seven of Diamonds
 * Eight of Diamonds
-------------------
     Hand Value: 15
Dealer wins! You lose $0.
Player Chips: $130


Do you want to play again? (Y or N) N


Your chip payout: $130!
